<a href="https://colab.research.google.com/github/luegl/keras-distinguish-own-cat-from-others-model/blob/main/preprocess_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install the library with pip

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.2/922.2 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import libraries

In [ ]:
import os
import cv2
import random
from ultralytics import YOLO
from tqdm import tqdm
import shutil

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


This script uses YOLOv8 to detect and crop cat images, saving them in categorized folders

In [ ]:
model = YOLO("yolov8n.pt")

input_folder = "/content/drive/MyDrive/datasets/dataset_fla_other/images"
output_folder = "/content/drive/MyDrive/datasets/dataset_fla_other/preprocessed_images"

for category in ["flaekli", "other_cats"]:
    os.makedirs(os.path.join(output_folder, category), exist_ok=True)

category_mapping = {
    "flaekli": "flaekli",
    "other_cats": "other_cats"
}

for category, target_folder in category_mapping.items():
    category_path = os.path.join(input_folder, category)
    output_category_path = os.path.join(output_folder, target_folder)

    if not os.path.exists(category_path):
        continue

    print(f"Processing images in category: {category}")

    existing_files = [f for f in os.listdir(output_category_path) if f.endswith(".jpg") or f.endswith(".png")]
    existing_numbers = [int(f.split('_')[-1].split('.')[0]) for f in existing_files if f.split('_')[-1].split('.')[0].isdigit()]
    start_index = max(existing_numbers, default=0) + 1

    img_files = [f for f in os.listdir(category_path) if f.endswith(".jpg") or f.endswith(".png")]

    with tqdm(total=len(img_files), desc=f"Processing: {category}", ncols=100) as pbar:
        for img_name in img_files:
            img_path = os.path.join(category_path, img_name)

            img = cv2.imread(img_path)
            if img is None:
                print(f"Error loading {img_path}")
                os.remove(img_path)
                continue

            results = model.predict(source=img, conf=0.4, classes=[15])

            if results[0].boxes:
                boxes = results[0].boxes.xyxy.tolist()
                for box in boxes:
                    x_min, y_min, x_max, y_max = map(int, box)
                    cropped_img = img[y_min:y_max, x_min:x_max]

                    cropped_img_path = os.path.join(output_category_path, f"{target_folder}_{start_index}.jpg")
                    cv2.imwrite(cropped_img_path, cropped_img)
                    start_index += 1
            else:
                os.remove(img_path)
                print(f"No object detected, deleting {img_path}")

            pbar.update(1)


Processing images in category: flaekli


Processing: flaekli:   0%|                                                  | 0/178 [00:00<?, ?it/s]


0: 640x480 1 cat, 172.3ms
Speed: 6.7ms preprocess, 172.3ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 480)


Processing: flaekli:   1%|▏                                         | 1/178 [00:00<01:02,  2.83it/s]


0: 640x640 1 cat, 228.4ms
Speed: 7.6ms preprocess, 228.4ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)


...

Processing: other_cats:  99%|████████████████████████████████████▊| 396/398 [01:51<00:00,  4.09it/s]


0: 640x640 1 cat, 258.9ms
Speed: 10.0ms preprocess, 258.9ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)


Processing: other_cats: 100%|████████████████████████████████████▉| 397/398 [01:51<00:00,  3.73it/s]


0: 544x640 1 cat, 201.1ms
Speed: 7.4ms preprocess, 201.1ms inference, 1.3ms postprocess per image at shape (1, 3, 544, 640)


Processing: other_cats: 100%|█████████████████████████████████████| 398/398 [01:52<00:00,  3.55it/s]


This code ensures both flaekli and other_cats folders contain the same number of images by removing excess images from the larger folder.

In [ ]:
flaekli_files = [f for f in os.listdir(os.path.join(output_folder, "flaekli")) if f.endswith(".jpg") or f.endswith(".png")]
other_cats_files = [f for f in os.listdir(os.path.join(output_folder, "other_cats")) if f.endswith(".jpg") or f.endswith(".png")]

print("Files in 'flaekli' folder:", flaekli_files)
print("Files in 'other_cats' folder:", other_cats_files)

flaekli_count = len(flaekli_files)
other_cats_count = len(other_cats_files)

print(f"flaekli count: {flaekli_count}, other_cats count: {other_cats_count}")

if flaekli_count != other_cats_count:
    target_count = min(flaekli_count, other_cats_count)

    if flaekli_count > target_count:
        excess_files = random.sample(flaekli_files, flaekli_count - target_count)
        for file in excess_files:
            os.remove(os.path.join(output_folder, "flaekli", file))

    elif other_cats_count > target_count:
        excess_files = random.sample(other_cats_files, other_cats_count - target_count)
        for file in excess_files:
            os.remove(os.path.join(output_folder, "other_cats", file))

    flaekli_files = [f for f in os.listdir(os.path.join(output_folder, "flaekli")) if f.endswith(".jpg") or f.endswith(".png")]
    other_cats_files = [f for f in os.listdir(os.path.join(output_folder, "other_cats")) if f.endswith(".jpg") or f.endswith(".png")]

    if flaekli_count < other_cats_count:
        for i in range(target_count - flaekli_count):
            file_to_copy = random.choice(other_cats_files)
            shutil.copy(os.path.join(output_folder, "other_cats", file_to_copy),
                        os.path.join(output_folder, "flaekli", file_to_copy))
    elif other_cats_count < flaekli_count:
        for i in range(target_count - other_cats_count):
            file_to_copy = random.choice(flaekli_files)
            shutil.copy(os.path.join(output_folder, "flaekli", file_to_copy),
                        os.path.join(output_folder, "other_cats", file_to_copy))

print("Images have been processed, balanced, and saved!")

Files in 'flaekli' folder: ['flaekli_1.jpg', 'flaekli_2.jpg', 'flaekli_3.jpg', 'flaekli_4.jpg', 'flaekli_5.jpg', 'flaekli_6.jpg', 'flaekli_7.jpg', 'flaekli_8.jpg', 'flaekli_9.jpg', 'flaekli_10.jpg', 'flaekli_11.jpg', 'flaekli_12.jpg', 'flaekli_13.jpg', 'flaekli_14.jpg', 'flaekli_15.jpg', 'flaekli_16.jpg', 'flaekli_17.jpg', 'flaekli_18.jpg', 'flaekli_19.jpg', 'flaekli_20.jpg', 'flaekli_21.jpg', 'flaekli_22.jpg', 'flaekli_23.jpg', 'flaekli_24.jpg', 'flaekli_25.jpg', 'flaekli_26.jpg', 'flaekli_27.jpg', 'flaekli_28.jpg', 'flaekli_29.jpg', 'flaekli_30.jpg', 'flaekli_31.jpg', 'flaekli_32.jpg', 'flaekli_33.jpg', 'flaekli_34.jpg', 'flaekli_35.jpg', 'flaekli_36.jpg', 'flaekli_37.jpg', 'flaekli_38.jpg', 'flaekli_39.jpg', 'flaekli_40.jpg', 'flaekli_41.jpg', 'flaekli_42.jpg', 'flaekli_43.jpg', 'flaekli_44.jpg', 'flaekli_45.jpg', 'flaekli_46.jpg', 'flaekli_47.jpg', 'flaekli_48.jpg', 'flaekli_49.jpg', 'flaekli_50.jpg', 'flaekli_51.jpg', 'flaekli_52.jpg', 'flaekli_53.jpg', 'flaekli_54.jpg', 'flaekli_